<a href="https://colab.research.google.com/github/Naveensadanandan/Personal_fitness_coach/blob/main/Fitness_coach_project_RAGpipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document processing or embedding creation

In [1]:
import os
import requests

github = "https://raw.githubusercontent.com/Naveensadanandan/Personal_fitness_coach/main/data/"

file_names = ["ncert_textbook_fitness.pdf","human-nutrition-text.pdf","JN - UpperLower Program.pdf"]

save_dir = "Data/"

if not os.path.exists(save_dir):
  print(f"file path doesnt exist, downloading")
  os.makedirs(save_dir, exist_ok=True)

  for file_name in file_names:

    base_url = github + file_name

    response = requests.get(base_url)

    if response.status_code == 200:
      with open(os.path.join(save_dir,file_name),"wb") as f:
        f.write(response.content)
      print(f"file has been saved as {file_name}")
    else:
      print(f"[INFO] failed to downlaod : status code {response.status_code}")
else:
  print(f"Directory already exists, skipping download")

file path doesnt exist, downloading
file has been saved as ncert_textbook_fitness.pdf
file has been saved as human-nutrition-text.pdf
file has been saved as JN - UpperLower Program.pdf


In [2]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 59.8 MB/s eta 0:00:00


In [3]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text:str) -> str:
  cleaned_text = text.replace("\n"," ").strip()
  return cleaned_text

def read_write_pdf(filepath: str) -> list[dict]:
  combined_list = []
  for file_name in os.listdir(filepath):
    filepath_for_pdf = os.path.join(filepath,file_name)
    if os.path.isfile(filepath_for_pdf):
      doc = fitz.open(filepath_for_pdf)

      for page_number , page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text)
        combined_list.append({"source" : file_name,
                                "page_number": page_number,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_setence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4, # 1 token = ~4 characters
                                "text": text})
  return combined_list

In [4]:
pages_and_texts = read_write_pdf(filepath=save_dir)
pages_and_texts[:2]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[{'source': 'ncert_textbook_fitness.pdf',
  'page_number': 0,
  'page_char_count': 95,
  'page_word_count': 19,
  'page_setence_count_raw': 1,
  'page_token_count': 23.75,
  'text': 'Health and Physical  Education Textbook for Class IX pre pages.indd   1 18-11-2016   4:25:29 PM'},
 {'source': 'ncert_textbook_fitness.pdf',
  'page_number': 1,
  'page_char_count': 1889,
  'page_word_count': 327,
  'page_setence_count_raw': 9,
  'page_token_count': 472.25,
  'text': 'First Edition November 2016 Kartika 1938 PD  5T BS ©\tNational Council of Educational  Research and Training, 2016 ` 135.00 Printed on 80 GSM paper with NCERT  watermark Published at the Publication Division  by the Secretary, National Council of  Educational Research and Training,   Sri Aurobindo Marg, New Delhi 110 016  and ................................... ISBN 978-93-5007-819-8 OFFICES OF THE PUBLICATION   DIVISION, NCERT NCERT Campus Sri Aurobindo Marg New  Delhi 110 016\t Phone :  011-26562708 108, 100 Feet Road  Hosd

In [5]:
import random
random.sample(pages_and_texts, k = 5)

[{'source': 'human-nutrition-text.pdf',
  'page_number': 970,
  'page_char_count': 877,
  'page_word_count': 146,
  'page_setence_count_raw': 5,
  'page_token_count': 219.25,
  'text': 'Learning Activities  Technology Note: The second edition of the Human  Nutrition Open Educational Resource (OER) textbook  features interactive learning activities.\xa0 These activities are  available in the web-based textbook and not available in the  downloadable versions (EPUB, Digital PDF, Print_PDF, or  Open Document).  Learning activities may be used across various mobile  devices, however, for the best user experience it is strongly  recommended that users complete these activities using a  desktop or laptop computer and in Google Chrome.  \xa0 An interactive or media element has been  excluded from this version of the text. You can  view it online here:  http://pressbooks.oer.hawaii.edu/ humannutrition2/?p=485  An interactive or media element has been  excluded from this version of the text. You

In [6]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df

,source,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count,text
0,ncert_textbook_fitness.pdf,0,95,19,1,23.75,Health and Physical Education Textbook for Cl...
1,ncert_textbook_fitness.pdf,1,1889,327,9,472.25,First Edition November 2016 Kartika 1938 PD 5...
2,ncert_textbook_fitness.pdf,2,2187,364,17,546.75,"Foreword Many factors like, physical, psycholo..."
3,ncert_textbook_fitness.pdf,3,373,62,2,93.25,As an organisation committed to systemic refor...
4,ncert_textbook_fitness.pdf,4,902,129,8,225.50,Textbook Development Committee Members Bharati...
...,...,...,...,...,...,...,...
1506,human-nutrition-text.pdf,1203,1676,252,18,419.00,39. Exercise 10.2 & 11.3 reused “Egg Oval Food...
1507,human-nutrition-text.pdf,1204,1617,254,10,404.25,Images / Pixabay License; “Pumpkin Cartoon Ora...
1508,human-nutrition-text.pdf,1205,1715,261,13,428.75,Flashcard Images Note: Most images in the fla...
1509,human-nutrition-text.pdf,1206,1733,268,13,433.25,ShareAlike 11. Organs reused “Pancreas Organ ...


In [7]:
file_names = ["ncert_textbook_fitness.pdf","human-nutrition-text.pdf","JN - UpperLower Program.pdf"]
#since every document page number doesnt start with 1 we need to change page number for evaluation in the future.
page_start = {"ncert_textbook_fitness.pdf" : 12,
              "human-nutrition-text.pdf" : 41,
              "JN - UpperLower Program.pdf" : 0}
for key, value in page_start.items():
  print(key,value)

ncert_textbook_fitness.pdf 12
human-nutrition-text.pdf 41
JN - UpperLower Program.pdf 0


In [8]:
df_copy = df.copy()
for key, value in page_start.items():
  df_copy.loc[df_copy["source"] == key, "page_number"] = df_copy["page_number"] - value
df_copy

,source,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count,text
0,ncert_textbook_fitness.pdf,-12,95,19,1,23.75,Health and Physical Education Textbook for Cl...
1,ncert_textbook_fitness.pdf,-11,1889,327,9,472.25,First Edition November 2016 Kartika 1938 PD 5...
2,ncert_textbook_fitness.pdf,-10,2187,364,17,546.75,"Foreword Many factors like, physical, psycholo..."
3,ncert_textbook_fitness.pdf,-9,373,62,2,93.25,As an organisation committed to systemic refor...
4,ncert_textbook_fitness.pdf,-8,902,129,8,225.50,Textbook Development Committee Members Bharati...
...,...,...,...,...,...,...,...
1506,human-nutrition-text.pdf,1162,1676,252,18,419.00,39. Exercise 10.2 & 11.3 reused “Egg Oval Food...
1507,human-nutrition-text.pdf,1163,1617,254,10,404.25,Images / Pixabay License; “Pumpkin Cartoon Ora...
1508,human-nutrition-text.pdf,1164,1715,261,13,428.75,Flashcard Images Note: Most images in the fla...
1509,human-nutrition-text.pdf,1165,1733,268,13,433.25,ShareAlike 11. Organs reused “Pancreas Organ ...


In [9]:
df_copy.describe()

,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count
count,1511.000000,1511.000000,1511.000000,1511.000000,1511.000000
mean,465.829914,1269.018531,221.774322,11.028458,317.254633
std,367.792908,654.278173,116.622741,7.283101,163.569543
min,-41.000000,0.000000,1.000000,1.000000,0.000000
25%,118.500000,818.500000,141.000000,5.000000,204.625000
50%,411.000000,1327.000000,229.000000,11.000000,331.750000
75%,788.500000,1701.500000,293.500000,16.000000,425.375000
max,1166.000000,3452.000000,627.000000,47.000000,863.000000


# Using Spacy to sentensize our text

In [10]:
from spacy.lang.en import English
nlp = English()
nlp.add_pipe("sentencizer")

In [11]:
pages_and_texts = df_copy.to_dict(orient = "records")

In [12]:
for item in tqdm(pages_and_texts):
  item["sentences"] = list(nlp(item["text"]).sents)

  item["sentences"] = [str(sentense) for sentense in item["sentences"]]

  item["spacy_sentence_count"] = len(item["sentences"])

  0%|          | 0/1511 [00:00<?, ?it/s]

In [13]:
random.sample(pages_and_texts, k=1)

[{'source': 'human-nutrition-text.pdf',
  'page_number': 107,
  'page_char_count': 2280,
  'page_word_count': 379,
  'page_setence_count_raw': 16,
  'page_token_count': 570.0,
  'text': 'in the endocrine system are the pituitary, thyroid, parathyroid,  adrenals, thymus, pineal, pancreas, ovaries, and testes. The glands  secrete hormones, which are biological molecules that regulate  cellular processes in other target tissues, so they require  transportation by the circulatory system. Adequate nutrition is  critical for the functioning of all the glands in the endocrine system.  A protein deficiency impairs gonadal-hormone release, preventing  reproduction. Athletic teenage girls with very little body fat often  do not menstruate. Children who are malnourished usually do not  produce enough growth hormone and fail to reach normal height  for their age group. Probably the most popularized connection  between nutrition and the functions of the endocrine system is  that unhealthy dietary p

In [14]:
from os.path import split
#chunking sentence togother
split_size_for_chunking = 10
def split_list(input_list : list,
               split_size : int = split_size_for_chunking) -> list[list]:
               chunk_list = [input_list[i:i+split_size] for i in range(0,len(input_list),split_size)]
               return chunk_list

test_list = list(range(25))

split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [15]:
for item in tqdm(pages_and_texts):
  item["sentence_chunks"] = split_list(input_list = item["sentences"], split_size= split_size_for_chunking)
  item["chunk_count"] = len(item["sentence_chunks"])

  0%|          | 0/1511 [00:00<?, ?it/s]

In [16]:
random.sample(pages_and_texts, k=1)

[{'source': 'human-nutrition-text.pdf',
  'page_number': 714,
  'page_char_count': 1664,
  'page_word_count': 253,
  'page_setence_count_raw': 14,
  'page_token_count': 416.0,
  'text': 'percent of the population meets their nutrient need is the EAR, and  the point at which 97 to 98 percent of the population meets their  needs is the RDA. The UL is the highest level at which you can  consume a nutrient without it being too much—as nutrient intake  increases beyond the UL, the risk of health problems resulting from  that nutrient increases.  Source: Dietary Reference Intakes Tables and Application. The  National Academies of Science, Engineering, and Medicine.\xa0 Health  and  Medicine  Division.  http://nationalacademies.org/HMD/ Activities/Nutrition/SummaryDRIs/DRI-Tables.aspx.  Accessed  November 22, 2017.  The Acceptable Macronutrient Distribution Range (AMDR) is the  calculated range of how much energy from carbohydrates, fats,  and protein is recommended for a healthy diet adequat

## Creating an attribute dictionary for each chunks

In [17]:
import re

chunks = []
for item in tqdm(pages_and_texts):
  for chunk in item["sentence_chunks"]:
    chunk_dict = {}
    chunk_dict["source"] = item["source"]
    chunk_dict["page_number"] = item["page_number"]

    joined_sentence = "".join(chunk).replace("  "," ").strip()
    joined_sentence = re.sub(r'\.([A-Z])', r'. \1', joined_sentence)

    chunk_dict["sentence_chunk"] = joined_sentence

    # Get some stats on our chunks
    chunk_dict["chunk_char_count"] = len(joined_sentence)
    chunk_dict["chunk_word_count"] = len([word for word in joined_sentence.split(" ")])
    chunk_dict["chunk_token_count"] = len(joined_sentence) / 4 # 1 token = ~4 chars

    chunks.append(chunk_dict)

  0%|          | 0/1511 [00:00<?, ?it/s]

In [18]:
random.sample(chunks, k=1)

[{'source': 'human-nutrition-text.pdf',
  'page_number': 917,
  'sentence_chunk': 'Updated January 2015. Accessed December 4, 2017. Middle Age | 917',
  'chunk_char_count': 65,
  'chunk_word_count': 11,
  'chunk_token_count': 16.25}]

In [19]:
chunks[20]["source"]

'ncert_textbook_fitness.pdf'

In [20]:
df = pd.DataFrame(chunks)
df.head()

,source,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,ncert_textbook_fitness.pdf,-12,Health and Physical Education Textbook for Cla...,92,16,23.00
1,ncert_textbook_fitness.pdf,-11,First Edition November 2016 Kartika 1938 PD 5T...,1858,296,464.50
2,ncert_textbook_fitness.pdf,-10,"Foreword Many factors like, physical, psycholo...",1543,235,385.75
3,ncert_textbook_fitness.pdf,-10,Experiential learning activities for acquiring...,605,91,151.25
4,ncert_textbook_fitness.pdf,-9,As an organisation committed to systemic refor...,367,56,91.75


In [21]:
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,2522.00,2522.00,2522.00,2522.00
mean,450.75,742.84,115.85,185.71
std,370.34,448.76,72.68,112.19
min,-41.00,12.00,3.00,3.00
25%,115.00,347.00,52.00,86.75
50%,354.50,739.00,115.00,184.75
75%,788.75,1097.00,172.00,274.25
max,1166.00,2353.00,357.00,588.25


## Deleting chunks whaving less than minimum token count

In [22]:
min_token_count = 30
pages_and_chunks_mintoken = df[df["chunk_token_count"] > min_token_count].to_dict( orient = "records")
len(pages_and_chunks_mintoken)

2325


## Check the link for details about pre trained models
[Semantic Search Models](https://sbert.net/docs/sentence_transformer/pretrained_models.html#semantic-search-models)

In [23]:
pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 7.8 MB/s eta 0:00:00


In [24]:
from sentence_transformers import SentenceTransformer
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

embedding_model = SentenceTransformer("multi-qa-mpnet-base-cos-v1", device = device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
#saving encoding

%%time

for item in tqdm(pages_and_chunks_mintoken):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/2325 [00:00<?, ?it/s]

CPU times: user 47.3 s, sys: 575 ms, total: 47.9 s
Wall time: 55.2 s


In [26]:
# lets embedd our data

text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_mintoken]

In [27]:
text_chunks_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=16, # you can experiment to find which batch size leads to best results
                                               convert_to_tensor=True)
text_chunks_embeddings

tensor([[ 0.0609, -0.0351, -0.0092,  ..., -0.0087, -0.0518, -0.0260],
        [-0.0080, -0.0325, -0.0305,  ...,  0.0041, -0.0058, -0.0262],
        [ 0.0493, -0.0224, -0.0399,  ..., -0.0058,  0.0140, -0.0549],
        ...,
        [ 0.1130,  0.0103, -0.0298,  ..., -0.0493, -0.0182, -0.0342],
        [ 0.1279, -0.0084, -0.0175,  ..., -0.0590, -0.0145, -0.0393],
        [ 0.0994, -0.0102, -0.0121,  ..., -0.0353, -0.0328, -0.0436]],
       device='cuda:0')

In [28]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_mintoken)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [29]:
text_chunks_embeddings.shape

torch.Size([2325, 768])

# Write a query and use semantic search for finding related topics

In [30]:
from sentence_transformers import SentenceTransformer,util
import torch
from time import perf_counter as timer


device = "cuda" if torch.cuda.is_available() else "cpu"

embedding_model = SentenceTransformer("multi-qa-mpnet-base-cos-v1", device = device)

query = "which workout is best for upperbody"

print(f"query : {query}")

query_embedding = embedding_model.encode(query,convert_to_tensor=True, device = device)
start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b = text_chunks_embeddings)[0]
end_time = timer()

print(f"[INFO] Time taken to get scores on {len(text_chunks_embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

# 4. Get the top-k results (we'll keep top 5)
top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

query : which workout is best for upperbody
[INFO] Time taken to get scores on 2325 embeddings: 0.02121 seconds.


torch.return_types.topk(
values=tensor([0.6070, 0.6066, 0.6035, 0.6021, 0.6020], device='cuda:0'),
indices=tensor([598, 596, 600, 597, 599], device='cuda:0'))

In [31]:
text_chunks[2276]

'This graphic shows the percentage of food-secure and food-insecure households in the United States during the year 2010. 1.\xa0Hunger in America: 2016 United States Hunger and Poverty Facts. World Hunger Education Service. Retrieved from http://www.worldhunger.org/articles/ Learn/us_hunger_facts.htm. Accessed April 15, 2018. Food Insecurity | 1133'

In [32]:
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    return wrapped_text

In [33]:
text_chunks[2]

'Experiential learning activities for acquiring skills for healthy living are made an integral part of the book. NCERT appreciates the hard work done by the Textbook Development Committee. I am highly thankful to Professor Saroj Yadav, Dean (Academic) and Project Coordinator of National Population Education Project (NPEP) and Adolescence Education Programme (AEP) for her continuous effort to develop this material in the present form. Several experts and teachers contributed towards the finalisation of this book. We are grateful to them and their institutions. pre pages.indd  3 18-11-2016  4:25:29 PM'

In [34]:
def print_top5(query:str, embedding_model = embedding_model):
  query_embedding = embedding_model.encode(query, convert_to_tensor=True, device =  device)
  dot_scores = util.dot_score(a=query_embedding, b =text_chunks_embeddings )[0]
  top_results = torch.topk(dot_scores, k =5)

  for score , idx in zip(top_results[0],top_results[1]):
    print(f"score : {score}")
    print(f"text")
    print(print_wrapped(text_chunks[idx]))
    # print_wrapped(pages_and_chunks_mintoken[idx])
    print(f"source : {pages_and_chunks_mintoken[idx]['source']}")
    print(f"page_number : {pages_and_chunks_mintoken[idx]['page_number']}")

In [36]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
    """
    Embeds a query with model and returns top k scores and indices from embeddings.
    """

    # Embed the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Get dot product scores on embeddings
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    if print_time:
        print(f"[INFO] Time taken to get scores on ({len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

    scores, indices = torch.topk(input=dot_scores,
                                 k=n_resources_to_return)

    return scores, indices

In [37]:
def print_top5(query:str, embedding_model = embedding_model):
  scores, indices =  retrieve_relevant_resources(query = query,
                                embeddings = text_chunks_embeddings,
                                model=embedding_model)

  for score , idx in zip(scores,indices):
    print(f"score : {score}")
    print(f"text")
    print(print_wrapped(text_chunks[idx]))
    # print_wrapped(pages_and_chunks_mintoken[idx])
    print(f"source : {pages_and_chunks_mintoken[idx]['source']}")
    print(f"page_number : {pages_and_chunks_mintoken[idx]['page_number']}")

In [45]:
print_top5(query = "how often do you need to workout")

[INFO] Time taken to get scores on (2325 embeddings: 0.00010 seconds.
score : 0.5935137271881104
text
Image by Allison Calabrese / CC BY 4.0 Physical Activity Recommendations The CDC
along with the American College of Sports Medicine (ACSM) have evidence based
 recommendations and guidelines for individuals to follow in order to obtain or
maintain a healthy lifestyle. Adults should get at least 150 minutes of
moderate- intensity aerobic physical activity or 75 minutes of vigorous-
intensity aerobic physical activity each week. In addition to aerobic physical
activity, it is recommended that adults do muscle strengthening activities on
each major muscle group two or three times each week. Adults also are
recommended by the ACSM to do flexibility exercises at least two to three times
a week to improve range of motion. To learn more about these guidelines visit
the CDC website at https://health.gov/paguidelines/guidelines/adults.aspx and
the ACSM website at http://www.acsm.org/about-acsm/

In [39]:
retrieve_relevant_resources(query = "how often do you need to workout",
                            embeddings = text_chunks_embeddings,
                            model=embedding_model)

[INFO] Time taken to get scores on (2325 embeddings: 0.00010 seconds.


(tensor([0.5935, 0.5854, 0.4975, 0.4926, 0.4808], device='cuda:0'),
 tensor([1984, 1389,  614, 1988, 1272], device='cuda:0'))

In [44]:
pip install -U bitsandbytes

In [41]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [73]:
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 15 GB


In [74]:
# Note: the following is Gemma focused, however, there are more and more LLMs of the 2B and 7B size appearing for local use.
if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
    use_quantization_config = True
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
    use_quantization_config = False
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit or float16 precision.")
    use_quantization_config = False
    model_id = "google/gemma-7b-it"

print(f"use_quantization_config set to: {use_quantization_config}")
print(f"model_id set to: {model_id}")

GPU memory: 15 | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.
use_quantization_config set to: False
model_id set to: google/gemma-2b-it


In [75]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available
from transformers import BitsAndBytesConfig
model_id = "google/gemma-2b-it"
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)

if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
    attn_implementation = "flash_attention_2"
else:
    attn_implementation = "sdpa" # scaled dot product attention
print(f"Using attention implementation: {attn_implementation}")

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)
llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = model_id,
                                                 torch_dtype = torch.float16,
                                                 quantization_config = quantization_config if use_quantization_config else None,
                                                 attn_implementation = attn_implementation,
                                                 low_cpu_mem_usage=False)

if not use_quantization_config:
    llm_model.to("cuda")

Using attention implementation: sdpa


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [76]:
input_text = "explain newtons first law"

chat_template = [{
    "role" : "user",
    "content" : input_text
}]

prompt = tokenizer.apply_chat_template(conversation=chat_template,
                                       tokenize = False,
                                       add_generation_prompt=True
                                       )

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = llm_model.generate(**input_ids,max_new_tokens = 256)
# print(outputs[0])
output_decoded = tokenizer.decode(outputs[0])

In [77]:
output_decoded

"<bos><bos><start_of_turn>user\nexplain newtons first law<end_of_turn>\n<start_of_turn>model\nSure, here's a summary of Newton's First Law:\n\n**Newton's First Law:**\n\n> An object at rest will remain at rest, and an object in motion will continue in a straight line with a constant velocity, unless acted upon by an external force.\n\n**Key Points:**\n\n* This law applies to all objects, regardless of their mass.\n* It tells us that the net force acting on an object must be zero for the object to remain at rest or move with a constant velocity.\n* It also tells us that the direction of the object's motion will remain the same, unless acted upon by an external force.\n\n**Examples:**\n\n* A ball at rest will remain at rest.\n* A car driving down a highway will continue to move in a straight line at a constant speed, unless it is acted upon by an external force, such as a wind or a hill.\n* A stone thrown in the air will follow a curved path due to the force of gravity.\n\n**Consequences

In [78]:
questions = [
    "What are the best exercises for building muscle?",
    "How can I improve my flexibility?",
    "What’s the ideal workout routine for fat loss?",
    "How many days a week should I work out?",
    "How can I stay motivated to exercise regularly?",
    "What are the benefits of strength training for women?",
    "How much cardio should I do each week for heart health?",
    "What’s the best time of day to do cardio?",
    "How can I build strength without lifting weights?",
    "Should I do cardio before or after strength training?",
    "What’s the best diet for muscle gain?",
    "How can I manage my protein intake while on a plant-based diet?",
    "Is intermittent fasting good for weight loss?",
    "How many calories should I consume daily to lose weight?",
    "What are some healthy pre-workout meals?",
    "How important is rest for muscle recovery?",
    "What are the best exercises to prevent lower back pain?",
    "How can I improve my post-workout recovery?",
    "How do I know if I’m overtraining?",
    "What should I do if I get injured during a workout?",
    "How should I structure a gym workout?",
    "How can I use resistance bands in my workout?",
    "What are some good alternatives to using gym machines?",
    "Should I lift heavier weights or do more reps?",
    "What are some common mistakes to avoid when lifting weights?",
    "How do I set realistic fitness goals?",
    "How can I track my progress in strength training?",
    "What’s a good workout routine for a beginner?",
    "How do I improve my running endurance?",
    "How long does it take to see results from working out?",
    "What’s the best workout for someone with knee problems?",
    "How can I stay fit while working a desk job?",
    "What exercises should I avoid if I have shoulder pain?",
    "How can I stay fit without access to a gym?",
    "What’s the best way to lose belly fat?"
]


In [82]:
def prompt_formatter(query:str,
                     context : list):
  context = "\n -".join([item["sentence_chunk"] for item in context])
  base_prompt = """Based on the following context items, please answer the query.
                  Give yourself room to think by extracting relevant passages from the context before answering the query.
                  Don't return the thinking, only return the answer.
                  Make sure your answers are as explanatory and actionable as possible.
                  Use the following examples as reference for the ideal answer style.
                  \nExample 1:
                  Query: How can I lose weight effectively?
                  Answer: To lose weight effectively, you need to create a calorie deficit by consuming fewer calories than you burn. A combination of regular physical activity and a balanced diet can help achieve this. Start by incorporating both cardio and strength training exercises into your routine, as these help burn calories and build muscle. Focus on nutrient-dense foods such as vegetables, lean proteins, and whole grains, while reducing your intake of processed foods and added sugars. Aim for gradual, sustainable weight loss of 1-2 pounds per week, and remember that consistency is key to long-term success.
                  \nExample 2:
                  Query: What are the best exercises to build core strength?
                  Answer: To build core strength, it's important to target a variety of muscles in your midsection, including the rectus abdominis, obliques, and lower back muscles. Some of the most effective exercises include planks, bicycle crunches, Russian twists, and leg raises. Additionally, compound movements like squats and deadlifts engage your core and help build overall strength. Aim to incorporate core-specific exercises 2-3 times per week as part of a well-rounded fitness routine.
                  \nExample 3:
                  Query: How can I improve my cardiovascular endurance?
                  Answer: Improving cardiovascular endurance involves consistent aerobic exercise that challenges your heart and lungs. Activities like running, cycling, swimming, and rowing are excellent for building endurance. Start by aiming for 30 minutes of moderate-intensity cardio 3-5 times per week. Gradually increase the duration and intensity of your workouts as your endurance improves. Interval training, which alternates between short bursts of high-intensity activity and periods of recovery, can also help enhance cardiovascular endurance efficiently.
                  \nNow use the following context items to answer the user query:
                  {context}
                  \nRelevant passages: <extract relevant passages from the context here>
                  User query: {query}
                  Answer:"""
  base_prompt = base_prompt.format(context = context, query = query)

  dialog_template = [{
      "role" : "user",
      "content" : base_prompt
  }]

  prompt = tokenizer.apply_chat_template(conversation=dialog_template,
                                           tokenize=False,
                                           add_generation_prompt=True)

  return prompt


In [83]:
def ask(query: str,
        temperature: float=0.7,
        max_new_tokens:int=256,
        format_answer_text=True,
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
    """

    # RETRIEVAL
    # Get just the scores and indices of top related results
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=text_chunks_embeddings)

    # Create a list of context items
    context_items = [pages_and_chunks_mintoken[i] for i in indices]

    # Add score to context item
    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu()

    # AUGMENTATION
    # Create the prompt and format it with context items
    prompt = prompt_formatter(query=query,
                              context=context_items)

    # GENERATION
    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate an output of tokens
    outputs = llm_model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)

    # Decode the tokens into text
    output_text = tokenizer.decode(outputs[0])

    # Format the answer
    if format_answer_text:
        # Replace prompt and special tokens
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "")

    # Only return the answer without context items
    if return_answer_only:
        return output_text

    return output_text

In [94]:
query = random.choice(questions)
print(f"Query: {query}")
ask(query="best sources of  protein",
    temperature=0.2,
    return_answer_only=False)

Query: What are some healthy pre-workout meals?
[INFO] Time taken to get scores on (2325 embeddings: 0.00009 seconds.


'Sure, here are the relevant passages from the context:\n\n- **Dietary Sources of Protein** states that protein is a nutrient that is important for maintaining a healthy body and should be consumed in a variety of forms.\n\n\n- **Protein Quality** states that high-quality protein contains all the essential amino acids in the proportions needed by the human body.\n\n\n- **Sources of Dietary Protein** provides a table comparing the nutritional value of different protein sources, including lean meats, seafood, plant-based proteins, and dairy products.'